In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import os
import re
import shutil
import string

In [5]:
#import data
features = pd.read_csv('nbme-score-clinical-patient-notes/features.csv')
patient_notes =pd.read_csv('nbme-score-clinical-patient-notes/patient_notes.csv')
sample_submission = pd.read_csv('nbme-score-clinical-patient-notes/sample_submission.csv')
test = pd.read_csv('nbme-score-clinical-patient-notes/test.csv')
train =pd.read_csv('nbme-score-clinical-patient-notes/train.csv')

In [6]:
#creating merged train data
merged = pd.merge(train,patient_notes, on = 'pn_num')
trainData = merged
trainData

,id,case_num_x,pn_num,feature_num,annotation,location,case_num_y,pn_history
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],0,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],0,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,['chest pressure'],['203 217'],0,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",0,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],0,HPI: 17yo M presents with palpitations. Patien...
...,...,...,...,...,...,...,...,...
14295,95333_912,9,95333,912,[],[],9,Stephanie madden is a 20 year old woman compla...
14296,95333_913,9,95333,913,[],[],9,Stephanie madden is a 20 year old woman compla...
14297,95333_914,9,95333,914,['photobia'],['274 282'],9,Stephanie madden is a 20 year old woman compla...
14298,95333_915,9,95333,915,['no sick contacts'],['421 437'],9,Stephanie madden is a 20 year old woman compla...


In [7]:
#tensor flow trial
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '\r\n' , ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [8]:
def text_standardization(text):
    # tokenize the text by replacing punctuation and numbers with spaces and lowercase all words
    punc_list = string.punctuation
    t = str.maketrans(dict.fromkeys(punc_list, ""))
    text = text.lower()
    #text = text.replace('\r\n' , ' ')
    text = text.translate(t)
    return text

In [9]:
trainData['pn_history'][0]

'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'

In [10]:
##developing text standardization code

# lowercase = tf.strings.lower(trainData['pn_history'][0])
# stripped_html = tf.strings.regex_replace(lowercase, '\r\n' , ' ')
# final = tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')
# final

#print(string.punctuation)
# lowercase = trainData['pn_history'][0].lower()
# stripped_html = lowercase.replace('\r\n' , ' ')
# punc_list = string.punctuation

# t = str.maketrans(dict.fromkeys(punc_list, ""))
# text = stripped_html.translate(t)
# #text = text.split()
# text

#testing
# cleanedText = custom_standardization(trainData['pn_history'][0])
# cleanedText

In [26]:
text = [text_standardization(trainData['pn_history'][0])]
text

['hpi 17yo m presents with palpitations patient reports 34 months of intermittent episodes of heart beatingpounding out of my chest 2 days ago during a soccer game had an episode but this time had chest pressure and felt as if he were going to pass out did not lose conciousness of note patient endorses abusing adderall primarily to study 13 times per week before recent soccer game took adderrall night before and morning of game denies shortness of breath diaphoresis fevers chills headache fatigue changes in sleep changes in visionhearing abdominal paun changes in bowel or urinary habits  pmhx none rx uses friends adderrall fhx mom with thyroid disease dad with recent heart attcak all none immunizations up to date shx freshmen in college endorses 34 drinks 3 nights  week on weekends denies tabacco endorses trying marijuana sexually active with girlfriend x 1 year uses condoms']

In [27]:
# text =text.split()
# text

In [28]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(text)
X_train_counts.shape

(1, 106)

In [29]:
#text

In [30]:
X_train_counts

<1x106 sparse matrix of type '<class 'numpy.int64'>'
	with 106 stored elements in Compressed Sparse Row format>

In [31]:
count_vect.vocabulary_.get(u'17yo')

1

In [33]:
xarray = X_train_counts.toarray()
xarray

array([[1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 3, 2, 1,
        1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3,
        1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
        1, 6, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2,
        1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 1, 4, 1]])

In [38]:
unique = np.array(X_train_counts)
print(unique)

  (0, 51)	1
  (0, 1)	1
  (0, 77)	1
  (0, 104)	4
  (0, 71)	1
  (0, 73)	2
  (0, 81)	1
  (0, 2)	2
  (0, 59)	1
  (0, 67)	6
  (0, 55)	1
  (0, 36)	1
  (0, 50)	2
  (0, 14)	1
  (0, 70)	2
  (0, 61)	1
  (0, 20)	2
  (0, 27)	1
  (0, 8)	1
  (0, 33)	1
  (0, 87)	2
  (0, 43)	3
  (0, 47)	2
  (0, 10)	1
  (0, 35)	1
  :	:
  (0, 40)	1
  (0, 58)	1
  (0, 91)	1
  (0, 31)	1
  (0, 25)	1
  (0, 13)	1
  (0, 9)	1
  (0, 53)	1
  (0, 97)	1
  (0, 26)	1
  (0, 85)	1
  (0, 41)	1
  (0, 22)	1
  (0, 32)	1
  (0, 63)	1
  (0, 68)	1
  (0, 102)	1
  (0, 89)	1
  (0, 96)	1
  (0, 57)	1
  (0, 83)	1
  (0, 5)	1
  (0, 44)	1
  (0, 105)	1
  (0, 24)	1
